In [1]:
import pymolpro
import os

In [2]:
xyzdir = 'G2-molecule-selection'
projectdir = 'Molpro projects'
if not os.path.isdir(projectdir):
    os.mkdir(projectdir)
molecules = []
for file in os.listdir(xyzdir):
    if file.endswith(".xyz"):
        molecules.append(file[:-4])


In [3]:
bases = ['cc-pVDZ',
         'cc-pVTZ',
         ]
methods = {
    'UHF': 'uhf',
    'RHF': 'rhf',
    'UMP2': 'uhf;ump2',
    'RMP2': 'rhf;rmp2',
}

In [10]:
projects = {}


def project_name(molecule, method, basis):
    return molecule + "-" + method + "-" + basis


for basis in bases:
    for method_name, method_implementation in methods.items():
        for molecule in molecules:
            name = project_name(molecule, method_name, basis)
            projects[name] = pymolpro.Project(name, location=projectdir)
            with open(os.path.join(xyzdir, molecule + ".xyz"), 'r') as initial_xyz:
                with open(molecule + ".xyz", 'w') as final_xyz:
                    for l in initial_xyz.readlines():
                        final_xyz.write(' '.join(l.split()[:4]) + '\n')
            projects[name].import_file(molecule + ".xyz")
            os.remove(molecule + ".xyz")
            projects[name].write_input(f"""
      geometry={molecule}.xyz
      basis,{basis}
      {method_implementation}
      """)

In [5]:
from multiprocessing.dummy import Pool
from operator import methodcaller

with Pool(processes=8) as pool:
    pool.map(methodcaller('run', backend='local', wait=True),
             projects.values(), 1)

assert pymolpro.all_completed(projects.values()) and pymolpro.no_errors(projects.values())

In [6]:
import pandas

for basis in bases:
    df = pandas.DataFrame({
        method: {molecule: projects[project_name(molecule, method, basis)].energies()[-1] for molecule in molecules} for
        method in methods.keys()
    })
    df.index.name = basis
    print(df)


                UHF         RHF        UMP2        RMP2
cc-pVDZ                                                
h2coh   -114.423204 -114.418839 -114.728507 -114.726455
no      -129.261309 -129.254714 -129.565601 -129.565890
ch       -38.272603  -38.268778  -38.358035  -38.355752
c2h5     -78.604144  -78.599732  -78.875873  -78.873855
oh       -75.393383  -75.389528  -75.541284  -75.538779
ch2      -38.863227  -38.863227  -38.978799  -38.978799
ch3      -39.563800  -39.559635  -39.694458  -39.692408
c2h3     -77.396709  -77.389499  -77.626734  -77.632298
c3h7    -117.645517 -117.640999 -118.059438 -118.057492
clo     -534.255979 -534.250303 -534.530142 -534.526933
sih3    -290.625045 -290.623591 -290.728152 -290.727212
on2     -183.682275 -183.682275 -184.200533 -184.200532
so      -472.279567 -472.330077 -472.594344 -472.627011
ch3o    -114.423579 -114.419647 -114.728754 -114.726760
nh2      -55.566996  -55.562735  -55.711701  -55.709257
sh      -398.088958 -398.084727 -398.202010 -398

In [7]:

# [(name, project.errors()) for name, project in projects.items()]


In [8]:
# print(projects['sh-UMP2-cc-pVDZ'].errors())

In [9]:
# print(projects['sh-UMP2-cc-pVDZ'].out)